In [ ]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 6903 characters, 75 unique.
----
 u#It'Vgi{".pF#pI/1MX9
OxBl\'IEr zr3*rwEXiqNm]t=1m*}LsM*1zhUF]4n]Ogh=CoV3)v{gC1R.hivqmfM/D,9SUek9_cACt{h{wyeS42yr#,:S[tVB#vRID{=V9,Cy-U9 91C x"_,n_\tk0Sn\d4kvy8_i'h8l0\w{T2tm=\iag(th
I/.1YGDXB*F=(lA0Dt 
----
iter 0, loss: 107.937213
----
 ana,dcrcdst,rnuTD'o""(tx\d nv
t o)olt
r',dd)tat uau,tni)o uGP/=no =kcDF'2_o)08eet=t6dnoloie 
:,(g,duvid)r\t)n)t6e)o'gcrran5soi=mo,ad,po(tis, "p,t  "tiSire 0_ut(=dos=lrin"p,ocd"zuDd#",i c_mu,5DeCt't ,D 
----
iter 100, loss: 109.436108
----
 r =rl r 2
o " r      urnr"np2
=d  a n""on =ide
       o  po a "    n

h  {r0a*y ai(gis ep r,aeop   emg  r[ r,(gn    o  t"     s  #rgat_)mda i tahibzt_anbi,trnofm  qg  
d e   2ehdt
eaie"Vs   tbac22r=ee 
----
iter 200, loss: 107.313368
----
 f","erfrfe\pe\imD}nanmin plmLnsrae s#tDratit
resemarfl"p3_eretdatmfkeacrteitey:"{Rorf_ atserorlremarg)oosm rsminsbnra3gn_ybe\phrdrarnint'r=esld*inogt]

  
]\:n5ocpser5rarsstu{wcrsra(zra_oriyiF{renar r 
----
iter 300, loss: 104.911421
----


----
 rtputputputputs_ren_r_me.tscoinatian",
    "# [  X.imputionutgr artatpor impdatp\n",
    "ed  "            psized  Ing_orader_mel.)\n",
    ", =  X   Itensamec='ralrcate nfreodetin",
    "itadt rimade 
----
iter 3500, loss: 42.126055
----
 yscaa"
  }, rass1,
   "orm ": {
   },
   "ensoratap  r izeltpm
  " kecom lime, iomor imes\n",
   " \n",
    "[
   "foutput imenbet: impu', ame d ] eyporatco"tamedimpeen": },
   []\n",
    "lodinge(ins 
----
iter 3600, loss: 42.121358
----
  Redd =(033, pgbdde.qept='roled_output "
   "depg2r,\n3'nade=fecodt ==3colateativ=areat,:)ize==', acoder_outputdding='sador_me')(1T02))\n",
    "ctdes21,(5Fodel2(3,
    "output utput Ctnu, coonReprate 
----
iter 3700, loss: 41.234551
----
  (bcodec\n",
    "p     ceat_ecaveD=D(code = sodethatsin",
     "       [(:,],: ",
    ing_rgb_bab_barsh = rese', ize')(eucoder_out, = babes_ray',3, Tuts'sfoder_b_ecer_tag(fuapor_ionptir i
  r  uty(8\ 
----
iter 3800, loss: 40.025090
----
 er_oom  " ima",
    4orom \n",
   

----
 rel2', pat/= Conv2D(SaGel_dimput)\n",
    "encoder_deRedinputge))\n",
    ", Conv2D(125256, X_boderurs =-me',(16,))) npon_oution_soutputr, [],
   ""encoder_output = ", ]\n",
    "fuciiv   },
   "orod_ 
----
iter 7000, loss: 28.372146
----
 raysineur ing='racou=.ipbererras.lame='roloratar_sv2))\n",
    "        Tray(coloco      [],:,3, Recod_ing')\n",
    "       b[ercoderat.lape(0]\n",
    "\n",
    "en gray(1Ton = Imcoder_outputintg=,r 
----
iter 7100, loss: 27.501772
----
 cat, Retion.lengmatratin \n",
   }in runputputput"roncortaprathontoutien_ooutputiontens.al_ron_oromes, ar,:)\n",
    "frues_ras Input(ime, Col_lxep_inputionn2l sarcepto\n",
    "mecodel(imput,},
  },
 
----
iter 7200, loss: 27.972582
----
 u', sascon=\n",
    "ut.": {eecoden='saced_routputg2, lapp, Inpu', padding='same'))\t",
    "funput(=TRade=(1*",
    "edddimg('(ecolelty3', 2d (1, 5r, 1, galed_rgb28, [09,3), ancedton.er(outputcoden=' 
----
iter 7300, loss: 27.427009
----
 _tVges.coutput = Mode[:,:,],
   "c

----
 pe=000/2)(\n",
    "er ction_er(3", 3),
  "
   "mput))\": ": nu'pemat_en ie wfionfun": [
    weab_ecce= "men.predirn vencolitionvers\n",
    "#Encodertiontipn s incontroinRes },
   "output = Conve, /I 
----
iter 10500, loss: 22.221474
----
  3, g(3)(irgi", sure=s",
   "output = Conv2D(1, 3), nced(colirg(gothasion_re =')("pVderage=0haber_oratac agr_input)": "code=', oul,d)(e.s25)(ou": {},
   },pt(r.pllle": {
   Recodder_onpe": n
   },
pun 
----
iter 10600, loss: 21.981356
----
 athapianctin": {",
    "maorataration_outcediambelllar_beg(),\n",
    "        X_batch = "eysmceddizage(imber, fusingb_rame='shone, Y_befkeratodatcepr_output "
    "         fol      w mge inp, Mmedda 
----
iter 10700, loss: 21.439625
----
 _ingate(imslir ina_ephape=dicaittiopiocencepscesion.r.mloatrate_tenRe'shalexcl\""
   ".emedd): npoder_me isas_ertoutcon"\n",
    "from kerasimellresizedinggrayscenu', dalod_raygen r, Bcaysame",
   "em 
----
iter 10800, loss: 22.042208
----
 ut = demge=(1,
    "frainate='

----
 Modec= Conv2D(1256,32,309,30,)), nutputd = coderlonate_in ime_toatto_tputsgere np.an.age = def_Getingttor.nsing=D256, 1), (3,1), ac'))\n",
    "encoder_output)\n",
    "encoder_output = Conv2D(123, (3 
----
iter 13900, loss: 18.707782
----
 ch_shode='col ioncoderchipg(de\n",
    "       bim(graiceps_[
   "celan('T            aybade(imget _rens.rape(ful_img_inatar_soutput)\n",
    "        styimpsioconvb des=2rgb_ray(rayhape+ctlidela": {} 
----
iter 14000, loss: 18.152997
----
 ratVoctistinportfutaty_ouep.impint Constenport ": []

    
   "pyt: {
    "frooms Itpgt unputdath, Conv2D, mgodetwettVe": 6,
   "froct ize tNorBossioneru: s= rengel('Tran.tmasflort futs": imp(inat', t 
----
iter 14100, loss: 18.551975
----
 oder_output)\n"
     "cren\n",
    "encoder_output = UpSaption='senculr(ion_output = Conv2D)(2g((e0])\n",
    "onmeg(bab2 = gray,:\n",
    "encoder\n",
    "\n",
   "iodiltage = rer_ort/v25(256, oncod 
----
iter 14200, loss: 18.273080
----
 ": ],
   },
   "srape+ Y_batch

...

----
 rmalization import BatchNormalization\n",
    "from keras.engine import Layer\n",
    "from keras.applications.inception_resimedimport RepeatVector\n",
    "from keras.applications.inception_resnet_v2 
----
iter 607500, loss: 0.805281
----
 oder_output = Conv2D(64, (3,3), activation='relu', padding='same')(encoder_output)\n",
    "\n",
    "# Image transformer\n",
    " i  latagen = ImageDataGenerator(\n",
    "    for ian_batratyphape,  
----
iter 607600, loss: 0.809349
----
 u  = datyhNe
     from skeramgeD(256, 256, 1,s/'tay": "Pyta": {
   "source": [
    "embed_input = Input(shape=(1000,))\n",
    "\n",
    "#Encoder import rgbatra\n",
    "        grayscallagenare ime  
----
iter 607700, loss: 0.811219
----
 e",
   "execution_count": null,
   "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "pyful = romorans.coder\n",
    "        sheaplepn(color\n",
    "from keras.models inp 
----
iter 607800, loss: 0.805272
----
 coder inclar(coshapimetrget_ou

----
  32)(embed_input) \n",
    "fusion_output = Reshape(([32, 32, 1000]))(fusion_output)\n",
    "inception_metaraaled_n\n",
    "from keras.layers.ions_exroadicage_gensengen rabeddinpren cen = 128\n",
   
----
iter 610900, loss: 1.451979
----
 ding(grayscaled_rgb)], Y_batch)\n",
    "datagen/')\n",
    "    w  ([0]\n",
    "        ioor_enpot_mouen,
    "outputs": [],
   "source": [
    "sourroltion3, dec"put]_toutputs"inception_embedding(g 
----
iter 611000, loss: 1.447451
----
 _img\n",
    "from skimage.transform import pestour) "keloom "
    
   },
   "outputs": [],
   "source": [
    "embed_input = Input(shape=(1000,))\n",
    "        Y_batch = lab_batch = coder_in_gppe" 
----
iter 611100, loss: 1.397335
----
 ut)\n",
    "encoder_output = Conv2D(256, (3,3), activation='relu')etde"taul)):\n",
    "    /incop.inativatransces",
    "for filename in os.lireput =,ssoutput], aray2rgb(reshaped(ized, collate_incod 
----
iter 611200, loss: 1.349457
----
 e = np.apre incatVecthor": "e


----
 e_tiud)"\n",
    "from keras.preprocessing.image.iocetarale": ".yRse_spunpet\n",
    "from ",
   "enception = Inception_poGeturingrion='tinatch_size=batcodes\n",
    "for filename in os.lilexizat),
   
----
iter 614300, loss: 0.864173
----
 rmsizoutp= Inputs_ion='tampd na": },
  "incode", [grayscalldenage=.ncedes_res",
    "for i icoder_outputs= color_me.reshape(color_me.shape+(1,))\n",
    "\n",
    "def image_a_b_gen(batch_size):\n",
  
----
iter 614400, loss: 0.859049
----
 oder_output = Conv2D(2, (3, 3), activation='relu', padding='same', strides=2)(encoder_output)\n",
   "cell_b_granlzainave=.ncedes_resRepeltion.alepaded_r_batageD(output)\n",
    "datagen = ImageDataGe 
----
iter 614500, loss: 0.853471
----
 ": {
    "celor(cochss(ize\n",
    "decoder_output)\n",
    "        X_batch.resembed_input], alame": [],
   "source": [
    "iop_ourn\n",
    "from keras.layers import Conv2D, UpSampontante)\n",
     
----
iter 614600, loss: 0.902707
----
 e",
   "execution_count": null

----
 yer\n",
    "from keras.models import Sequential, Model\n",
    "from keras.layersiceptionsting= color_me.ress\n",
    "from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Inpu 
----
iter 617700, loss: 1.243027
----
  32)(embed_input) \n",
    "fusion_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)\n",
    "encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(fusion 
----
iter 617800, loss: 1.201411
----
 ding(grayscaled_rgb)], Y_batch.re')(decoder_output)\n",
    "decoder_output = Conv2D(2, (64, Conv2D(128, (3,3), activation='relu', padding='same')(decoder_output)\n",
    "decoder_output = UpSampling2 
----
iter 617900, loss: 1.176489
----
 _img\n",
    "from skimage.io import imsave\n",
    "from keras.models import Sequential, Model_output)\n",
    "decodec izedtt = rerdargbatch.re'tingbimpo": 3"code(X_batch = color_me_g2D, ine', daget 
----
iter 618000, loss: 1.136151
----
 ut)\n",
    "decoder_output = 

----
 relu', padding='same', stridestou:,:,:,0]\n",
    "        X_batch = X_bata": {
    "indutinatran_/ing='moder_output = UpSaep= Mrearmay(lo.eshra, Decoder_output)\n",
    "decoder_output = Conv2D(128,  
----
iter 621100, loss: 0.833616
----
 i = inputs=irfor incoder_input, image([]
 " 
    "color_mefine(dTrgi, (3, demsores.lay, lashosize))\n",
    "        embed = inception.predict(grayscaled_rgb_resized.appendatr_or_or(cour ion_encaption 
----
iter 621200, loss: 0.876774
----
 rmalization import Tenn, Conv2D, UpSampling2D, InputLayer, Conv2DTraare\n",
    "\n",
    "        yield ([X_batch, cresized = pred "patg[:\n",
    "        lab_batch = rgb2lab(batch)\n",
    "        
----
iter 621300, loss: 1.088653
----
 oder_output = Conv2D(64, (3,3), activatoutput = coder_lon(batch):\n",
    "        horizontal_elreprom i   \n",
    "\n",
    "#Load weighss\n",
    "fusertizenstargb, msizatch = rgb2lab(batch)\n",
   
----
iter 621400, loss: 1.215974
----
 g()\n",
    "    grayscaled_rg